In [1]:
import torch
import numpy as np
import os
import pytorch_lightning as pl

from torchphysics.problem import Variable
from torchphysics.setting import Setting
from torchphysics.problem.domain import Interval
from torchphysics.problem.condition import (DirichletCondition,
                                            DiffEqBoundaryCondition,
                                            DiffEqCondition)
from torchphysics.models.fcn import SimpleFCN
from torchphysics import PINNModule
from torchphysics.utils import grad

from torchphysics.setting import Setting

os.environ["CUDA_VISIBLE_DEVICES"] = "" # select GPUs to use

#pl.seed_everything(43) # set a global seed
torch.cuda.is_available()

False

In [2]:
# First define all parameters:
h_0 = 1.e-02 #m = 16 um
delta_h = 3.5e-03 #m = 10 um 
D = 1 #m = 100 mm 
L = np.pi*D # Länge von Gebiet
u_m = 3.5 #m/s 0.26
beta = 2.2*1e-08 # 2.2e-08 m^2/N
nu_0 = 1.5e-03 # Pa·s = 1.5 mPa·s
nu0 = 1.5e-04
nu1 = 1.8e-03
p_0 = 1e+5 # 1e+5 N/m^2 = 1 bar

In [3]:
# define h:
def h(x):
    return h_0 + delta_h * torch.cos(2*x/D) # here use x/D, instead of x? Or else
    #return h_0                                   # the function is almost constant.

# we can analytically compute h':
def h_x(x):
    return -2.0*delta_h/D * torch.sin(2*x/D) # x in [0,pi*D]
    #return 0

# create a data function, so we only need to evaluate h and h' once at the beginning
def data_fun(x):
    out_h = h(x)
    out_d_h = h_x(x)
    out = np.column_stack((out_h, out_d_h)) # column 0 : values of h
                                            # column 1 : values of h' 
    return out.astype(np.float32)

In [4]:
# define the function of the viscosity.
# Here we need torch.tensors, since the function will be evaluated in the pde.
# At the beginng the model will have values close to 0, 
# therefore the viscosity will also be close to zero. 
# This will make the pde condition unstable, because we divide by nu.
# For now set values smaller then 1e-06 to 1e-06 
def nu_func(nu,p):
    out = nu * torch.exp(beta * p)
    return torch.clamp(out, min=1e-06)

In [5]:
# Variables:
x = Variable(name='x',
             order=1,
             domain=Interval(0, np.pi*D),
             train_conditions={},
             val_conditions={})
nu = Variable(name='nu',
             order=1,
             domain=Interval(low_bound=nu0,
                             up_bound=nu1),
             train_conditions={},
             val_conditions={})
# Output:
p = 'p'

In [6]:
norm = torch.nn.MSELoss()

# periodic boundary -> use arbritray boundary conditon class:
nu_points = 10 # number of time-points for the boundary condition.
# The points will be in such order, that the first points will be 
# at the left x-boundary, and the last points at the right side. 
# This will be consitent over all iterations, therefore we can compute
# the specific index once:
index_left = range(0, nu_points)
index_right = range(nu_points, 2*nu_points)
def bcl_fun(p, x):
    #print('left', x[index_left]) # to check correct order of x
    return p-1#- p[index_right]) 

def bcr_fun(p, x):
    #print('left', x[index_left]) # to check correct order of x
    return p-1 

x.add_train_condition(DiffEqBoundaryCondition(bound_condition_fun=bcl_fun,
                                              name='bcl',
                                              norm=norm,
                                              dataset_size = [1,nu_points], 
                                              sampling_strategy='grid',
                                              boundary_sampling_strategy='lower_bound_only'))

x.add_train_condition(DiffEqBoundaryCondition(bound_condition_fun=bcr_fun,
                                              name='bcr',
                                              norm=norm,
                                              dataset_size = [1,nu_points], 
                                              sampling_strategy='grid',
                                              boundary_sampling_strategy='upper_bound_only'))

In [7]:
def pde(nu, p, x):
    # evaluate the viscosity
    nu = nu_func(nu,p)
    # implement the PDE:
    # 
    rs = 6*u_m*nu
    #out = h(x)*grad(grad(p,x),x) +3*h_x(x)*grad(p,x) - rs*h_x(x)/h(x)/h(x)
    p_x  = grad(p,x)
    p_xx = grad(p_x,x)
    out = - nu*p_xx + p_x + p - 2
    
    #print(torch.max(out))
    return out
   
train_cond = DiffEqCondition(pde=pde,
                             name='pde',
                             #data_fun=data_fun,
                             norm=norm,
                             weight=1.0,
                             dataset_size=[5000,10],
                             sampling_strategy = 'grid',
                             data_plot_variables=('x','nu'))


print('pde defined'

In [8]:
setup = Setting(variables=(x, nu),
                train_conditions={'pde': train_cond},
                val_conditions={},
                solution_dims={p: 1},
                n_iterations=180)

In [9]:
solver = PINNModule(model=SimpleFCN(variable_dims=setup.variable_dims,
                                    solution_dims=setup.solution_dims,
                                    depth=3,
                                    width=20),
                    optimizer=torch.optim.Adam,
                    lr=1.e-3)

In [10]:
#solver.model.layers
#solver.model.layers[-1].bias = torch.nn.Parameter(torch.tensor(0.2))
#solver.model.layers[-1].weight.data.fill_(0.1)

In [ ]:
trainer = pl.Trainer(gpus='-1' if torch.cuda.is_available() else None,
                     num_sanity_val_steps=0,
                     benchmark=True,
                     check_val_every_n_epoch=20,
                     log_every_n_steps=1,
                     max_epochs=2,
                     logger=False,
                     checkpoint_callback=False
                     )

trainer.fit(solver, setup)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name  | Type      | Params
------------------------------------
0 | model | SimpleFCN | 1.3 K 
------------------------------------
1.3 K     Trainable params
0         Non-trainable params
1.3 K     Total params
0.005     Total estimated model params size (MB)
C:\Users\inu2sh\Desktop\Torch-Physics\_venv\lib\site-packages\pytorch_lightning\trainer\data_loading.py:102: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\inu2sh\Desktop\Torch-Physics\_venv\lib\site-packages\pytorch_lightning\trainer\data_loading.py:102: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argumen

Training: 0it [00:00, ?it/s]

In [ ]:
# Use LBFGS at the end
solver.lr = 1.e-2
solver.optimizer = torch.optim.LBFGS
solver.optim_params = {'max_iter': 5}
trainer = pl.Trainer(gpus='-1' if torch.cuda.is_available() else None,
                     num_sanity_val_steps=0,
                     benchmark=True,
                     check_val_every_n_epoch=20,
                     log_every_n_steps=15,
                     max_epochs=1,
                     logger=True,
                     checkpoint_callback=False
                     )

trainer.fit(solver, setup)

In [ ]:
from torchphysics.utils.plot import _plot
solver = solver.to('cpu')
print('nu0= ',nu0,' nu1= ',nu1)
nu = 1.8e-5
if nu0<=nu and nu<=nu1:
    fig = _plot(model=solver.model, solution_name='p', plot_variables=[x], # or [x, nu] and 
                points=300, angle=[30, 30], dic_for_other_variables={'nu': nu}) # dic_for_... = {}
else:
    print('Ausserhalb des Trainingsbereiches')